In [1]:
import numpy as np
import pandas as pd

import zipfile


from theano import tensor

import fuel
import h5py
from fuel.datasets.hdf5 import H5PYDataset
from fuel.streams import DataStream
from fuel.schemes import SequentialScheme
from fuel.transformers import Flatten
from fuel.datasets.text import TextFile

from blocks.bricks import Linear, Logistic, Softmax
from blocks.bricks.cost import CategoricalCrossEntropy
from blocks.roles import WEIGHT
from blocks.graph import ComputationGraph
from blocks.filter import VariableFilter
from blocks.initialization import IsotropicGaussian, Constant
from blocks.algorithms import GradientDescent, Scale, AdaGrad, Adam
from blocks.extensions.monitoring import DataStreamMonitoring
from blocks.main_loop import MainLoop
from blocks.extensions import FinishAfter, Printing
from blocks.extensions.monitoring import TrainingDataMonitoring
from blocks_extras.extensions.plot import Plot

%matplotlib inline

In [2]:
filename = "./data/text8.zip"

def read_data(filename):
  f = zipfile.ZipFile(filename)
  for name in f.namelist():
    return f.read(name).split()
  f.close()
  
#words = read_data(filename)
#print 'Data size', len(words)

In [12]:
from fuel.datasets.base import Dataset

class WordWindow(Dataset):
    def __init__(self, **kwargs):
        self.provides_sources = ('features', 'targets')
         # for technical reasons
        self.axis_labels = None
        with self.open() as fh:
            self.corpus = fh.read().split(" ")
        
        super(WordWindow, self).__init__(**kwargs)
    
    def get_data(self, state=None, request=None):
        data = 
        return self.filter_sources(data)

    def open(self):
        return open('./data/small')
    
    def close(self,fh):
        fh.close()
    
dataset = WordWindow().get_data() 

SyntaxError: invalid syntax (<ipython-input-12-96e939baa811>, line 14)

In [10]:
from fuel.streams import DataStream
print dictionary
for data in DataStream(text_data).get_epoch_iterator():
    print data

{'</S>': 1, '<UNK>': 0}
([0, 0, 0, 0, 0, 1],)
([0, 0, 0, 0, 0, 1],)
([0, 0, 0, 0, 0, 1],)
([0, 0, 0, 0, 0, 1],)
([0, 0, 0, 0, 0, 1],)
([0, 0, 0, 0, 0, 1],)


In [ ]:
data_stream = DataStream(text_data,
     iteration_scheme=SequentialScheme(train_data.num_examples, batch_size=50))

In [ ]:
def makeGraph():
    Xs = tensor.matrix("features")
    y = tensor.lmatrix('targets')
    
    h1 = Linear(name='h1', input_dim=117, output_dim=70)
    a1 = Logistic().apply(h1.apply(Xs))
    h2 = Linear(name='h2', input_dim=70, output_dim=2)
    y_hat = Softmax().apply(h2.apply(a1))
    
    h1.weights_init = h2.weights_init = IsotropicGaussian(0.01)
    h1.biases_init = h2.biases_init = Constant(0)
    h1.initialize()
    h2.initialize()
    
    cost = CategoricalCrossEntropy().apply(y, y_hat)
    
    cg = ComputationGraph(cost)
    W1, W2 = VariableFilter(roles=[WEIGHT])(cg.variables)
    
    cost = cost + 0.005 * (W1 ** 2).sum() + 0.005 * (W2 ** 2).sum()
    cost.name = 'cost_with_regularization'
    
    mcr = MisclassificationRate().apply(y.argmax(axis=1), y_hat)
    mcr.name = "misclassification rate"
    
    return cg,(W1,W2),cost,mcr